In [1]:
import json
import requests
from config import lw_key
import pandas as pd
from datetime import date, timedelta, datetime
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

In [2]:
url= "https://api.worldweatheronline.com/premium/v1/weather.ashx?"
past_url = "https://api.worldweatheronline.com/premium/v1/past-weather.ashx?"
# query_url =f"{url}appid={api_key}&zip="

In [3]:
zipcode = input("Enter 5-digit Zipcode:")

Enter 5-digit Zipcode:21061


In [4]:
# zip_df = nomi.query_postal_code(zipcode)


In [5]:
# pandas apply pgeocode to ZIPCODE1
# md_zips = df['ZIPCODE1'].apply(nomi.query_postal_code)
# md_zips.head()

In [6]:
response = requests.get(f"{url}key={lw_key}&q={zipcode}&num_of_days=8&tp=24&cc=no&mca=no&aqi=yes&format=json").json()
response = response['data']


In [7]:
weather_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
}

weather_dict["Dates"]= ([response["weather"][i]["date"] for i in range(len(response["weather"]))])
weather_dict["Cloudcover"]=([response["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(response["weather"]))])
weather_dict["Humidity"]= ([response["weather"][i]["hourly"][0]["humidity"] for i in range(len(response["weather"]))])
weather_dict["PrecipInch"]= ([response["weather"][i]["hourly"][0]["precipInches"] for i in range(len(response["weather"]))])
weather_dict["Pressure"]= ([response["weather"][i]["hourly"][0]["pressure"] for i in range(len(response["weather"]))])
weather_dict["FeelsLike"]= ([response["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(response["weather"]))])
weather_dict["HeatIndex"]= ([response["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(response["weather"]))])
weather_dict["MaxTemp"]= ([response["weather"][i]["maxtempF"] for i in range(len(response["weather"]))])
weather_dict["MinTemp"]=([response["weather"][i]["mintempF"] for i in range(len(response["weather"]))])
weather_dict["SunHours"]=([response["weather"][i]["sunHour"] for i in range(len(response["weather"]))])
weather_dict["UVIndex"]= ([response["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(response["weather"]))])
weather_df= pd.DataFrame.from_dict(weather_dict,orient='index').transpose()
weather_df= weather_df.apply(pd.to_numeric, errors= 'ignore')
weather_df["TempDelta"] = weather_df.MaxTemp - weather_df.MinTemp
weather_df["BarChange"] =weather_df["Pressure"].pct_change()
weather_df["HeatChange"] =weather_df["HeatIndex"].pct_change()
weather_df["HumChange"] =weather_df["Humidity"].pct_change()
weather_df= weather_df.iloc[1:]
weather_df.head(10)


,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange
1,2021-04-16,68,62,0.0,1009,42,48,53,43,7.8,3,10,0.000992,-0.127273,-0.088235
2,2021-04-17,78,64,0.0,1011,47,50,54,46,9.1,2,8,0.001982,0.041667,0.032258
3,2021-04-18,48,61,0.0,1013,49,51,56,47,9.1,4,9,0.001978,0.020000,-0.046875
4,2021-04-19,47,59,0.0,1015,52,53,60,44,10.4,1,16,0.001974,0.039216,-0.032787
5,2021-04-20,18,60,0.0,1017,56,57,66,49,11.6,4,17,0.001970,0.075472,0.016949
6,2021-04-21,23,56,0.0,1015,51,54,58,49,10.4,4,9,-0.001967,-0.052632,-0.066667
7,2021-04-22,11,47,0.0,1022,46,50,56,43,11.6,3,13,0.006897,-0.074074,-0.160714


In [8]:
dates= input("Dates you had a migraine (YYYY-MM-DD):")

Dates you had a migraine (YYYY-MM-DD):2021-04-15


In [9]:
dates=dates.split(",")
dates= [i.strip() for i in dates]

In [10]:
for date in dates:
    enddate= ((datetime.strptime(date, '%Y-%m-%d')- timedelta(days=1)).strftime("%Y-%m-%d"))
    enddate.strip()
    print(enddate)

2021-04-14


In [11]:
history_dict ={
    "Dates":[],
    "Cloudcover":[],
    "Humidity":[],
    "PrecipInch":[],
    "Pressure":[],
    "FeelsLike":[],
    "HeatIndex":[],
    "MaxTemp":[],
    "MinTemp":[],
    "SunHours":[],
    "UVIndex":[],
    }

In [12]:
for date in dates:
    enddate = (datetime.strptime(date, '%Y-%m-%d')- timedelta(days=1)).strftime("%Y-%m-%d")
    responses = requests.get(f"{past_url}key={lw_key}&q={zipcode}&date={enddate}&enddate={date}&tp=24&mca=no&aqi=yes&format=json").json()
    responses = responses['data']


    history_dict["Dates"].append([responses["weather"][i]["date"] for i in range(len(responses["weather"]))])
    history_dict["Cloudcover"].append([responses["weather"][i]["hourly"][0]["cloudcover"] for i in range(len(responses["weather"]))])
    history_dict["Humidity"].append([responses["weather"][i]["hourly"][0]["humidity"] for i in range(len(responses["weather"]))])
    history_dict["PrecipInch"].append([responses["weather"][i]["hourly"][0]["precipInches"] for i in range(len(responses["weather"]))])
    history_dict["Pressure"].append([responses["weather"][i]["hourly"][0]["pressure"] for i in range(len(responses["weather"]))])
    history_dict["FeelsLike"].append([responses["weather"][i]["hourly"][0]["FeelsLikeF"] for i in range(len(responses["weather"]))])
    history_dict["HeatIndex"].append([responses["weather"][i]["hourly"][0]["HeatIndexF"] for i in range(len(responses["weather"]))])
    history_dict["MaxTemp"].append([responses["weather"][i]["maxtempF"] for i in range(len(responses["weather"]))])
    history_dict["MinTemp"].append([responses["weather"][i]["mintempF"] for i in range(len(responses["weather"]))])
    history_dict["SunHours"].append([responses["weather"][i]["sunHour"] for i in range(len(responses["weather"]))])
    history_dict["UVIndex"].append([responses["weather"][i]["hourly"][0]["uvIndex"] for i in range(len(responses["weather"]))])



In [13]:
history_df= pd.DataFrame.from_dict(history_dict,orient='index').transpose()
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')


In [14]:
history_df= history_df.apply(pd.Series.explode).reset_index(drop=True)
history_df= history_df.apply(pd.to_numeric, errors= 'ignore')
history_df.reset_index(drop=True)
history_df.head(10)

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex
0,2021-04-14,70,88,0.3,1016,51,52,54,50,6.6,3
1,2021-04-15,70,73,0.2,1008,53,54,60,50,7.8,3


In [15]:
history_df["TempDelta"] = history_df.MaxTemp - weather_df.MinTemp
history_df["BarChange"] =history_df["Pressure"].pct_change()
history_df["HeatChange"] =history_df["HeatIndex"].pct_change()
history_df["HumChange"] =history_df["Humidity"].pct_change()
history_df= history_df.iloc[1::2]
history_df.head(10)

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange
1,2021-04-15,70,73,0.2,1008,53,54,60,50,7.8,3,17.0,-0.007874,0.038462,-0.170455


In [17]:
ml_hist_df = history_df
ml_hist_df

,Dates,Cloudcover,Humidity,PrecipInch,Pressure,FeelsLike,HeatIndex,MaxTemp,MinTemp,SunHours,UVIndex,TempDelta,BarChange,HeatChange,HumChange
1,2021-04-15,70,73,0.2,1008,53,54,60,50,7.8,3,17.0,-0.007874,0.038462,-0.170455


# Machine Learning

In [ ]:
risk_data= []
for i in ml_hist_df["BarChange"]:
    if i <=5:
       risk_data.append("1")
    elif i>5 & i<8:
        risk_data.append("2")
    else: 
        risk_data.append("3")
risk_data          
            

In [ ]:
ml_hist_df["MigraineRisk"] = risk_data


In [ ]:
ml_hist_df=ml_hist_df.drop(columns =["Dates"])
ml_hist_df

In [ ]:
# Assign X (data) and y (target)
X = ml_hist_df.drop("MigraineRisk", axis=1)
y = ml_hist_df["MigraineRisk"]
print(X.shape, y.shape)

In [ ]:
#Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
#Fit on the training data
X_scaler = MinMaxScaler().fit(X_train)
X_scaler.get_params()

In [ ]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

In [ ]:
y_train.value_counts()

In [ ]:
y_test.value_counts()

In [ ]:
model = SVC(kernel="linear")
model

# w/o Grid Search

In [ ]:
model.fit(X_train, y_train)

In [ ]:
print('Test Acc: %.3f' % model.score(X_test, y_test))

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["1","2","3"]))

In [ ]:
print('Test Acc: %.3f' % model.score(X_test, y_test))

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["1", "2", "3"]))

# Grid Search

In [1]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

NameError: name 'GridSearchCV' is not defined

In [ ]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

In [ ]:
# List the best parameters for this dataset
print(grid.best_params_)

In [ ]:
# List the best score
print(grid.best_score_)

In [ ]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [ ]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

In [ ]:
predictions

In [ ]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                             target_names=["1", "2", "3"]))